In [4]:
import numpy as np 
import cv2
import matplotlib.pyplot as plt

# path to the database - change it if needed
path = 'D:\\Python\\MachineLearning\\MachineLearning\\week07\\data\\face_data\\face_data\\' 

ids = range(1, 16) # 15 people
states = ['centerlight', 'glasses', 'happy', 'leftlight', 
          'noglasses', 'normal', 'rightlight','sad', 
          'sleepy', 'surprised', 'wink' ]
prefix = 'subject'
surfix = '.png' #file extension is png

# open one picture to get the image's size
fn = prefix + '01.' + states[0] + surfix
im = cv2.imread(path + fn, 0)

h = im.shape[0] # hight 
w = im.shape[1] # width

D = h * w
N = len(states)*15 
print(N, D, h, w)

X = np.zeros((D, N))
Y = np.zeros(N)

# collect all data
cnt = 0 

# there are 15 people
for person_id in range(1, 16):
    for state in states:
        
        # get name of each image file
        fn = path + prefix + str(person_id).zfill(2) + '.' + state + surfix
        
        # open the file and read as grey image
        tmp = cv2.imread(fn, cv2.IMREAD_GRAYSCALE)
        
        # then add image to dataset X
        X[:, cnt] = tmp.reshape(D)
        Y[cnt] = states.index(state)
        
        cnt += 1 
Y = Y.astype(int)

165 77760 243 320


In [5]:
X.shape

(77760, 165)

In [6]:
# Doing PCA, note that each row is a datapoint
from sklearn.decomposition import PCA

# remain dim. k = 125 - change it! 
pca = PCA(n_components=125) 

# then apply to data X
pca.fit(X.T)

PCA(n_components=125)

###### Use PCA to find (norm) orthogonal bases  U

In [7]:
# then build projection matrix 
U = pca.components_.T
U.shape

(77760, 125)

###### Project X to less-dimention sub-space: Z = U^T . X^
####### Use ANN classifier to solve classification of Z 

In [8]:
Xhat = np.zeros((D, N))
x_mean = X.mean(1)
for i in range(N):
    Xhat[:,i] = X[:,i] - x_mean[:]
    
    
# Reduced dim. data Z (project of Xhat onto sub-space by Uk - bases)
Z = U.T.dot(Xhat)
Z.shape    

(125, 165)

###### Build ANN from numpy
####### Next, we build a Multi Layers Perceptron with 01 hidden layer, wich has dim.=100.
####### All of the codes are already shown in the lessons

In [9]:
import math


# one-hot coding
from scipy import sparse
def convert_labels(y, C):
    Y = sparse.coo_matrix((np.ones_like(y),
        (y, np.arange(len(y)))), shape = (C, len(y))).toarray()
    return Y

#softmax for multi-class
def softmax(V):
    e_V = np.exp(V - np.max(V, axis = 0, keepdims = True))
    #print(np.max(V, axis = 0, keepdims = True))
    Z = e_V / e_V.sum(axis = 0)
    return Z

#definition of ReLU, or you can use maximum directly
def ReLU(V):
    return np.maximum(V, 0)

# cost or loss function
def cost(Y, Yhat):
    return -np.sum(Y*np.log(Yhat))/Y.shape[1]


In [10]:
# Gradient Descent Loop
# To use this code easily, we put it to a method

def ANN_3layer_SolveClassification(X_train, Y_train, eta, max_count, num):
    X, Y = X_train, Y_train
    d, N = X_train.shape
    C = Y_train.shape[0]
    d1, d2 = num, C
    # make random data
    W1 = 0.01*np.random.randn(d, d1)
    b1 = np.zeros((d1, 1))
    W2 = 0.01*np.random.randn(d1, d2)
    b2 = np.zeros((d2, 1))

    for i in range(max_count + 1):
        ## Feedforward
        Z1 = np.dot(W1.T, X) + b1
        A1 = ReLU(Z1)
        Z2 = np.dot(W2.T, A1) + b2
        Yhat = softmax(Z2)
        
          
        # print loss after each 10 iterations
        if i % 100 == 0:
            loss = cost(Y, Yhat)
            print("iter %d, loss: %f" %(i, loss))
            #print(i)
        # backpropagation
        E2 = (Yhat - Y )/N
        dW2 = np.dot(A1, E2.T)
        db2 = np.sum(E2, axis = 1, keepdims = True)
        E1 = np.dot(W2, E2)
        E1[Z1 <= 0] = 0 # gradient of ReLU
        dW1 = np.dot(X, E1.T)
        db1 = np.sum(E1, axis = 1, keepdims = True)

        # Gradient Descent update
        W1 += -eta*dW1
        b1 += -eta*db1
        W2 += -eta*dW2
        b2 += -eta*db2

    return W1, W2, b1, b2


def predict(W1, W2, b1, b2, images):
    Z1 = np.dot(W1.T, images) + b1
    A1 = ReLU(Z1)
    Z2 = np.dot(W2.T, A1) + b2
    a = softmax(Z2)
    #return  np.argmax(Z2, axis=0) #
    return np.argmax(a, axis=0)

In [11]:
#80% for training set - You can change this rate by yourselves
M = (int)(Z.shape[1]*0.95)
X_train = Z[:, :M]
Y_train = convert_labels(Y[:M], 11)

print(X_train.shape)
print(Y_train.shape)

W1, W2, b1, b2 = ANN_3layer_SolveClassification(X_train, Y_train, 1e-6,4000, num = 400)

(125, 156)
(11, 156)
iter 0, loss: 47.634465
iter 100, loss: 14.487080
iter 200, loss: 8.505375
iter 300, loss: 5.409839
iter 400, loss: 3.612222
iter 500, loss: 2.513008
iter 600, loss: 1.816331
iter 700, loss: 1.324550
iter 800, loss: 1.012239
iter 900, loss: 0.775590
iter 1000, loss: 0.601243
iter 1100, loss: 0.482581
iter 1200, loss: 0.397437
iter 1300, loss: 0.337153
iter 1400, loss: 0.298495
iter 1500, loss: 0.270888
iter 1600, loss: 0.249788
iter 1700, loss: 0.233175
iter 1800, loss: 0.219875
iter 1900, loss: 0.208887
iter 2000, loss: 0.199665
iter 2100, loss: 0.191822
iter 2200, loss: 0.185049
iter 2300, loss: 0.179039
iter 2400, loss: 0.173764
iter 2500, loss: 0.168994
iter 2600, loss: 0.164795
iter 2700, loss: 0.161078
iter 2800, loss: 0.157741
iter 2900, loss: 0.154704
iter 3000, loss: 0.151922
iter 3100, loss: 0.149363
iter 3200, loss: 0.147008
iter 3300, loss: 0.144833
iter 3400, loss: 0.142814
iter 3500, loss: 0.140931
iter 3600, loss: 0.139182
iter 3700, loss: 0.137569
i

In [12]:
from sklearn.metrics import accuracy_score # for evaluating results

Y_pred_train = predict(W1, W2, b1, b2, X_train)
print('accuracy training data: ', accuracy_score(Y[:M], Y_pred_train))

X_val = Z[:, M:]
Y_val = convert_labels(Y[M:], 11)


Y_pred_val = predict(W1, W2, b1, b2, X_val)
print(X_val.shape, Y_pred_val.shape, 'accuracy validation data: ', accuracy_score(Y[M:], Y_pred_val))
print(Y[M:])
print(Y_pred_val)

accuracy training data:  0.9423076923076923
(125, 9) (9,) accuracy validation data:  0.2222222222222222
[ 2  3  4  5  6  7  8  9 10]
[0 3 6 6 6 1 2 2 1]


###### Classify with naive bayes model

In [13]:
from sklearn.naive_bayes import GaussianNB

NB_model = GaussianNB()
NB_model.fit(X_train.T, Y[:M].T)

Y_predict_nb = NB_model.predict(X_train.T)
accuracy_nb = accuracy_score(Y[:M].T, Y_predict_nb)
print("Train Accuracy score using Naive Bayes: ", accuracy_nb)

Y_predict_val_nb = NB_model.predict(X_val.T)
accuracy_val_nb = accuracy_score(Y[M:].T, Y_predict_val_nb)
print("Validation Accuracy score using Naive Bayes: ", accuracy_val_nb)


Train Accuracy score using Naive Bayes:  0.8717948717948718
Validation Accuracy score using Naive Bayes:  0.2222222222222222


###### Classify with MultiNomial Logistic (SoftMax) Regression model

In [16]:
from sklearn.linear_model import LogisticRegression

lorg=LogisticRegression(multi_class='multinomial',solver='sag', max_iter=5000, penalty=None)
lorg.fit(X_train.T,Y[:M].T)

Y_pred_softmax =lorg.predict(X_train.T)
accuracy_LSM = accuracy_score(Y[:M].T, Y_pred_softmax)
print("Train Accuracy score using MultiNomial Logistic Reg.: ", accuracy_LSM)

Y_pred_val_softmax =lorg.predict(X_val.T)
accuracy_val_LSM = accuracy_score(Y[M:].T, Y_pred_val_softmax)
print("Train Accuracy score using MultiNomial Logistic Reg.: ", accuracy_val_LSM)

Train Accuracy score using MultiNomial Logistic Reg.:  0.9423076923076923
Train Accuracy score using MultiNomial Logistic Reg.:  0.2222222222222222


###### Use MLPClassifier (Multi Layers Perceptron - Classifier) from Sklearn lib 

In [17]:
# Import Neural Network
from sklearn.neural_network import MLPClassifier
ann = MLPClassifier()

ann.fit(Z.T, Y)


MLPClassifier()

In [18]:
y_pred = ann.predict(Z.T)
y_pred

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10,  0,  1,  2,  3,  5,  5,
        6,  7,  8,  9, 10,  0,  1,  2,  3,  5,  5,  6,  7,  8,  9, 10,  0,
        1,  2,  3,  4,  7,  6,  7,  8,  9, 10,  0,  1,  2,  3,  4,  5,  6,
        7,  8,  9, 10,  0,  1,  2,  3,  5,  5,  6,  7,  8,  9, 10,  0,  1,
        2,  3,  5,  5,  6,  7,  8,  9, 10,  0,  5,  2,  3,  4,  5,  6,  7,
        8,  9, 10,  0,  1,  2,  3,  5,  5,  6,  7,  8,  9, 10,  0,  1,  2,
        3,  4,  5,  6,  7,  8,  9, 10,  0,  1,  2,  3,  4,  5,  6,  7,  8,
        9, 10,  0,  1,  2,  3,  4,  4,  6,  7,  8,  9, 10,  0,  1,  2,  3,
        4,  5,  6,  7,  8,  9, 10,  0,  1,  2,  3,  4,  4,  6,  7,  8,  9,
       10,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [19]:
Y

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10,  0,  1,  2,  3,  4,  5,
        6,  7,  8,  9, 10,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10,  0,
        1,  2,  3,  4,  5,  6,  7,  8,  9, 10,  0,  1,  2,  3,  4,  5,  6,
        7,  8,  9, 10,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10,  0,  1,
        2,  3,  4,  5,  6,  7,  8,  9, 10,  0,  1,  2,  3,  4,  5,  6,  7,
        8,  9, 10,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10,  0,  1,  2,
        3,  4,  5,  6,  7,  8,  9, 10,  0,  1,  2,  3,  4,  5,  6,  7,  8,
        9, 10,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10,  0,  1,  2,  3,
        4,  5,  6,  7,  8,  9, 10,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9,
       10,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [20]:
from sklearn.metrics import accuracy_score # for evaluating results
print(accuracy_score(Y, y_pred))

0.9454545454545454
